<a href="https://colab.research.google.com/github/DENGCPU/TensorFlow-Study/blob/main/%E8%B6%85%E5%8F%82%E6%95%B0%E8%B0%83%E8%8A%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 使用keras Turner为机器学习ML选择正确的超参数集

# 超参数是控制训练过程和ML模型拓扑的变量，会直接影响ML的性能
# 1、模型超参数：影响模型的选择，例如隐藏层的数量和宽度
# 2、算法超参数：影响学习算法的速度和质量，例如随机梯度下降SGD的学习率和K近邻KNN分类器的近邻数

In [11]:
# 使用Keras Tuner对图像分类执行超调
import tensorflow as tf
from tensorflow import keras

In [12]:
pip install -q -U keras-tuner

In [13]:
import keras_tuner as kt

In [14]:
# 下载并准备数据集
(img_train,label_train), (img_test,label_test)=keras.datasets.fashion_mnist.load_data()

# normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [15]:
# 定义超模型
def model_builder(hp):
  model =keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))
  # tune the number of units in the first Dense layer
  # choose an optimal value between 32-512
  hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
  model.add(keras.layers.Dense(units=hp_units,activation='relu'))
  model.add(keras.layers.Dense(10))

  # tune the learning rate for the optimizer
  # choose an optimal value from 0.01, 0.001, 0.0001
  hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])

# from_logits=True 表示模型的输出是logits（即未经过softmax或sigmoid激活的原始输出）。在这种情况下，损失函数内部会应用softmax激活，并计算交叉熵损失。如果你的模型输出已经是概率分布（例如，经过了softmax层），你应该设置from_logits=False
  model.compile(optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [17]:
# 实例化调节器并执行超调（使用Hyperband调节器
tuner=kt.Hyperband(model_builder,
                   objective='val_accuracy',
                   max_epochs=10,
                   factor=3,
                   directory='my_dir',
                   project_name='intro_to_kt')
# 创建回调，在验证损失达到特定值后提前停止训练
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

# 运行超参数搜索
tuner.search(img_train,label_train,epochs=50,validation_split=0.2,callbacks=[stop_early])
# 得到最佳的超参数
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Reloading Tuner from my_dir/intro_to_kt/tuner0.json

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.001.



In [18]:
# 训练模型
# build the model with the optimal hyperparameters and train it on data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    img_train,
    label_train,
    epochs=50,
    validation_split=0.2
)
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))+1
print('best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.4982 - accuracy: 0.8225 - val_loss: 0.4308 - val_accuracy: 0.8457
Epoch 2/50
1500/1500 [==============================] - 10s 7ms/step - loss: 0.3676 - accuracy: 0.8662 - val_loss: 0.3539 - val_accuracy: 0.8719
Epoch 3/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3272 - accuracy: 0.8804 - val_loss: 0.3477 - val_accuracy: 0.8748
Epoch 4/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3042 - accuracy: 0.8865 - val_loss: 0.3210 - val_accuracy: 0.8839
Epoch 5/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.2851 - accuracy: 0.8940 - val_loss: 0.3272 - val_accuracy: 0.8809
Epoch 6/50
1500/1500 [==============================] - 11s 7ms/step - loss: 0.2695 - accuracy: 0.8987 - val_loss: 0.3315 - val_accuracy: 0.8820
Epoch 7/50
1500/1500 [==============================] - 11s 7ms/step - loss: 0.2557 - accuracy: 0.9041 - val_loss: 0.3154 - val_ac

In [20]:
# 评估超模型
eval_result = model.evaluate(img_test,label_test)
print('[test loss, test accuracy]:', eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.5741 - accuracy: 0.8914
[test loss, test accuracy]: [0.5741127729415894, 0.8913999795913696]
